# 从 9702 Physics 考试真题中提取高频词汇表

在备课中，发现部分学生做A-Level物理考试中的计算题相对熟练，但是碰到 explain 和 define 就难以下手或答不到点子上。目前市面上也有部分国际高中物理词汇表。这篇 jupyter notebook 将统计词频和分类自动化，用量化的方式分析词汇表。

## 安装所需环境

1. 需要阅读PDF文件 (真题PDF请自行合并成一个文件) -> `PyPDF2`
2. 生成 corpus 需要对 text 进行 clean -> `cleantext` 和自编的 filter
3. 自然语言处理的 tokenise and lemmatise -> `nltk`
4. 利用现成的自然语言模型生成单词向量 word vector -> `gensim`
5. 利用单词向量分类单词 -> `sklearn`(`sklearn.cluster.KMeans`)
6. 自动化翻译管线 -> `tranlators`

In [ ]:
# pip install requirements
! pip install PyPDF2 clean-text nltk wordcloud gensim numpy sklearn translators
# nltk download contents
import nltk
nltk.download('popular')

In [ ]:
# load packages
# to load PDF and extract corpus
import PyPDF2
from cleantext import clean
from nltk import WordNetLemmatizer, tokenize, corpus

# to count word frequency
from collections import Counter

# to generate word cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

# to apply pre-trained nlp model
import gensim.downloader

# to use K-Means clustering
import numpy as np 
from sklearn.cluster import KMeans 

# to translate vocabulary list
import time
import translators as ts

## 载入PDF并提取语料

首先使用`PyPDF2`载入PDF文件

In [ ]:
reader = PyPDF2.PdfFileReader('physics_full.pdf')

将PDF转化为`plaintext`并移除下列内容:
- 数字
- 标点符号
- 其他非ASCII符号
- stopwords
- 非英语单词

最后将corpus保存在`physics_full.txt`文件中

Note: 作者使用约10000页材料，此步骤大约消耗10分钟。

In [ ]:
# open text file for saving processed corpus
f = open('physics_full.txt','w')
# load stop words
stopwords = corpus.stopwords.words('english')
# load word net lammatizer
wn = WordNetLemmatizer()
# load word set
words = set(corpus.words.words())

print('Start generating corpus')
for page_num in range(reader.getNumPages()):

    # report progress
    print('Page',page_num,'/',reader.getNumPages(),end='')
    
    # extract plaintext from PDF
    taster = reader.getPage(page_num).extractText()

    # cleaning
    taster = clean(taster, no_line_breaks=True, no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    
    # tokenise and remove stopwords and non-English words
    taster = tokenize.word_tokenize(taster)
    taster_token = [i for i in taster if i not in stopwords and len(i) > 2 and i in words]
    
    # lemmatise
    taster_lemma = [wn.lemmatize(word) for word in taster_token]

    # write into corpus file
    f.write(' '.join(taster_lemma)+' ')

    # remove tmp variables to save memory
    del taster, taster_token, taster_lemma
    print('',end='\r')

f.close()

del reader, f, stopwords, wn, words

## 统计词频

读取上步骤的语料，统计词频并导出词频前2000词到`counts_full.txt`文件。

另生成词频前2000词的`str list`以便后续分析。

In [ ]:
# read corpus from txt file
with open('physics_full.txt','r') as f:
    words = f.read().split()
    f.close()

# count word frequency
cnt = Counter(words)

# generate top 2000 words
word_list_2000 = sorted(cnt, key=cnt.get, reverse=True)[:2000]

# save word frequency (counts) to a csv file
with open('counts_full.csv','w') as f:
    for k, v in cnt.most_common():
        f.write(k + ',' + str(v) + '\n')
    f.close()

# save top 2000 words as a one word per line txt
with open('top_2000_word.txt','w') as f:
    for x in word_list_2000:
        f.writelines(x + '\n')

### 生成词云

大家喜闻乐见的词云 (宣传用)

挑选前200词并输出为2000x1000的透明底PNG图片。

In [ ]:
# initiate word could clss
wordcloud = WordCloud(width=2000,height=1000,max_font_size=400,background_color='rgba(255, 255, 255, 0)', mode='RGBA').generate(' '.join(words))

# save image
wordcloud.to_file('physics_full_wordcloud.png')

## 使用预训练模型生成单词向量

这里使用`fasttext-wiki-news-subwords-300`模型。根据 https://github.com/RaRe-Technologies/gensim-data ，此模型基于160亿token训练单词向量，并包含了Wikipedia 2017的数据。Wikipedia大概率会有相关物理单词的词条，因此使用此模型生成单词向量。

In [ ]:
# load word vector model
glove_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

print(glove_vectors)
print(glove_vectors['physics'])

生成一个`numpy`矩阵来保存词频前2000的单词向量。

Note: 也有可能此模型不包含个别单词的情况，目前只能先跳过。

In [ ]:
# load word list
with open('top_2000_word.txt','r') as f:
    word_list_2000 = []
    for line in f:
        word_list_2000.append(line.strip())

# empty array for word vectors
x = np.zeros((len(word_list_2000),len(glove_vectors['physics'])))

# assign word vectors to the array
for index, word in enumerate(word_list_2000):
    try:
        x[index,:] = glove_vectors[word]
    except:
        pass

## 使用单词向量聚类

利用预训练的K-Means算法将上步骤生成的单词向量分类成20类。此20类是考虑每个分类不高于100词为目标，方便学生背诵。

导出分组信息到`top_2000_word_group.txt`以便后续翻译。

In [ ]:
# set number of clusters
clusters = 24

# apply KMeans to the data
kmeans = KMeans(n_clusters=clusters)
kmeans.fit_predict(x)

# save grouping data to file
np.savetxt('top_2000_word_group.txt',kmeans.labels_)

### 简易的 K-Means 性能分析

使用 Sum of Squred Error (SSE) 来统计单词向量到中心点的距离之和。SSE 越大说明 cluster 越分散。SSE 和 cluster 数量制图，折线的异常低点是我们希望找到的最适合的 cluster 数量。如果此折线未见到明显的转折点，说明 word vector 未显示明显分类。

In [ ]:
# analyse performance of k-means clustering
score = {}
for clusters in range(1,30):
    kmeans = KMeans(n_clusters=clusters).fit(x)
    score[clusters] = kmeans.inertia_

plt.figure(0,figsize=(6,4))    
plt.scatter(list(score.keys()),list(score.values()))
plt.xlabel('Number of clusters')
plt.ylabel('SSE')

## 自动化翻译

这里简单介绍两种自动化翻译的方法。

1. 利用翻译API自动翻译单词表
2. Google translate 文档翻译

### 利用翻译API自动翻译单词表

2000个单词翻译起来也不是轻松的事情。最基础的自动化便是使用开放的翻译API来翻译单词。当然，翻译效果比较差也不符合语境。但是可以减少人工翻译的时间。

最后将`单词, 组别, 自动翻译`输出到`physics_full_group.csv`以便后续调整

In [ ]:
# load word list
with open('top_2000_word.dat','rb') as f:
    word_list_2000 = pickle.load(f)

# load grouping data
top_2000_word_group = np.loadtxt('top_2000_word_group.txt',delimiter=',')

# open final translated output
groupout = open('physics_full_group.csv','w')

# reset counter
counter = 0

print('Translate:')
for word,group in zip(word_list_2000,top_2000_word_group):

    # report translate progress
    print('Working on word',counter+1, '/',len(word_list_2000),end='')
    
    # request translate
    translation = ts.google(word,to_language='zh')

    # write into the output
    groupout.writelines(str(word) + ',' + str(int(group))+ ',' + str(translation) +'\n')

    counter+=1
    
    time.sleep(1)

    print('',end='\r')
    
groupout.close()

## Google translate 文档翻译

上一个方法对于本项目有一丝困难，因为各类翻译 API 在 request 200个单词的时候差不多就不再响应了。的确每次翻译单个单词会让 API 拒绝接受请求来降低流量。这里另辟蹊径，使用文档翻译来翻译单词表。

导出为 one word per line 的 txt 文件 `top_2000_word.txt`，然后上传到 https://translate.google.cn/?sl=en&tl=zh-CN&op=docs 后进行文档翻译。

翻译结果网页另存为 `top_2000_word_translated.txt` 后程序读取到 `word_list_2000_translated` 列表。

In [ ]:
# load translated txt file
with open('top_2000_word_translated.txt','r') as f:
    word_list_2000_translated = []
    for line in f:
        word_list_2000_translated.append(line.strip())

生成和上方法同样的 `单词, 组别, 自动翻译` 格式的 `physics_full_group.csv` 文件

In [ ]:
# open final translated output
groupout = open('physics_full_group.csv','w')

# load grouping data
top_2000_word_group = np.loadtxt('top_2000_word_group.txt',delimiter=',')

for word,group,translation in zip(word_list_2000,top_2000_word_group,word_list_2000_translated):
    # write into the output
    groupout.writelines(str(word) + ',' + str(int(group))+ ',' + str(translation) +'\n')

groupout.close()

## 总结

此套自动统计词频并分组脚本可以比较稳定的输出给定文件的高频词汇。利用预训练模型可以对高频词汇进行词汇分类 (word embedding)。对于大量的单词，可以调用翻译 API 或输出文档进行文档翻译，减轻工作量。

### 目前的不足和计划

- PDF处理

    - PDF可以不用提前进行合并，而是使用 `glob` module 来对目录进行搜索。

- NLP相关

    - 对 text 进行 cleaning 时移除 punctuations，使得句子信息丢失。如果不移除，语料可保留以句为单位的信息。对于未来的句子语义理解会有帮助。

    - Word embedding 的 performance 没有进行分析。

- ML相关

    - K-Means clustering 的 performance 没有分析。clustering 真的收敛了吗？
